In [22]:
import psycopg2
import os

stockhost = os.environ.get('STOCKS_DATABASE_LOCALHOST')
#stockhost = os.environ.get('STOCKS_DATABASE_HOST')
stockdb = os.environ.get('STOCKS_DATABASE_DB')
stockuser = os.environ.get('STOCKS_DATABASE_USER')
stockpwd = os.environ.get('STOCKS_DATABASE_PWD')

try:
    conn = psycopg2.connect(
        host=stockhost,
        user=stockuser,
        database=stockdb,
        password=stockpwd
    )
    conn.set_session(autocommit=True)
    
except psycopg2.Error as e:
    print("Error could not make connection to postgres database")
    print(e)
try:
    cur= conn.cursor()
except psycopg2.Error as e:
    print("Error could not get curser to postgres database")
    print(e)

In [25]:
# cur.execute('DROP TABLE companies');
try:
    cur.execute(
    "CREATE TABLE IF NOT EXISTS companies ( \
    symbol varchar PRIMARY KEY, \
    name varchar NOT NULL,\
    exchange varchar NOT NULL,\
    sector varchar,\
    summary varchar, \
    timestamp timestamp default current_timestamp);")
    
except psycopg2.Error as e:
    print("Error Issue creating a table")
    print(e)
    
# Alter table : https://www.postgresql.org/docs/9.1/sql-altertable.html

In [34]:
try:
    cur.execute(
    "CREATE TABLE IF NOT EXISTS dailystocks ( \
    date date, \
    symbol varchar REFERENCES companies , \
    open numeric NOT NULL,\
    high numeric NOT NULL,\
    low numeric NOT NULL,\
    close numeric NOT NULL,\
    volume numeric NOT NULL ,\
    splits numeric,\
    dividends numeric, \
    timestamp timestamp default current_timestamp, \
    PRIMARY KEY (date, symbol));")
    
except psycopg2.Error as e:
    print("Error Issue creating a table")
    print(e)

#    https://www.postgresql.org/docs/8.1/ddl-constraints.html
#    product_no integer REFERENCES products ON DELETE RESTRICT,
#    order_id integer REFERENCES orders ON DELETE CASCADE

In [35]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as  plt


msft = yf.Ticker('msft')
msft_data = msft.history(start="2020-09-01",interval='1d')
msft_data.reset_index(inplace=True)
msft_data['symbol']= 'msft'
msft_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,symbol
0,2020-08-31,226.407249,228.102806,223.724270,224.941086,28774200,0.0,0,msft
1,2020-09-01,224.921145,226.856082,223.843964,226.676559,25791200,0.0,0,msft
2,2020-09-02,227.374722,232.251952,226.756345,231.045105,34080800,0.0,0,msft
3,2020-09-03,228.671335,228.711224,214.398704,216.732590,58400300,0.0,0,msft
4,2020-09-04,214.538338,217.789820,204.654212,213.690552,59664100,0.0,0,msft


In [39]:
from sqlalchemy import create_engine
postgresqlpath='postgresql://' + stockuser+':'+stockpwd +'@' +stockhost +':5432/' + stockdb
engine = create_engine(postgresqlpath, echo=True)


In [49]:
msft_data.rename(columns={"Date": "date", "Open": "open", "High": "high", "Close": "close", "Low": "low", 
                          "Dividends": "dividends", "Volume": "volume", "Stock Splits": "splits"},inplace=True)

In [50]:
msft_data.head()

,date,open,high,low,close,volume,dividends,splits,symbol
0,2020-08-31,226.407249,228.102806,223.724270,224.941086,28774200,0.0,0,msft
1,2020-09-01,224.921145,226.856082,223.843964,226.676559,25791200,0.0,0,msft
2,2020-09-02,227.374722,232.251952,226.756345,231.045105,34080800,0.0,0,msft
3,2020-09-03,228.671335,228.711224,214.398704,216.732590,58400300,0.0,0,msft
4,2020-09-04,214.538338,217.789820,204.654212,213.690552,59664100,0.0,0,msft


In [64]:
msft_data.to_sql('dailystocks', con=engine , if_exists='append', index=False)

2021-02-13 12:52:12,265 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-02-13 12:52:12,266 INFO sqlalchemy.engine.base.Engine {'name': 'dailystocks'}
2021-02-13 12:52:12,270 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-13 12:52:12,276 INFO sqlalchemy.engine.base.Engine INSERT INTO dailystocks (date, open, high, low, close, volume, dividends, splits, symbol) VALUES (%(date)s, %(open)s, %(high)s, %(low)s, %(close)s, %(volume)s, %(dividends)s, %(splits)s, %(symbol)s)
2021-02-13 12:52:12,278 INFO sqlalchemy.engine.base.Engine ({'date': datetime.datetime(2020, 8, 31, 0, 0), 'open': 226.40724851008102, 'high': 228.1028063582178, 'low': 223.7242703107674, 'close': 224.9410858154297, 'volume': 28774200, 'dividends': 0.0, 'splits': 0, 'symbol': 'msft'}, {'date': datetime.datetime(2020, 9, 1, 0, 0), 'open': 224.92114539989774, 'high': 226.856082129216

In [62]:
try:
    cur.execute("INSERT INTO companies ( \
        symbol, \
        name,\
        exchange,\
        sector,\
        summary) \
        VALUES (%s,%s,%s,%s,%s);",
        ('msft', "Microsoft Company",'NASDAQ', "Technology",'This will be the summary field ....'))
except psycopg2.Error as e:
    print("Error Inserting rows")
    print(e)

In [63]:
try:
    cur.execute("SELECT * FROM companies;")
except psycopg2.Error as e:
    print("Error reading rows")
    print(e)
    
row = cur. fetchone()
while row:
    print(row)
    row=cur.fetchone()

('msft', 'Microsoft Company', 'NASDAQ', 'Technology', 'This will be the summary field ....', datetime.datetime(2021, 2, 13, 12, 51, 20, 6209))


In [1]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as  plt

tickers = yf.Tickers('msft aapl goog')

In [8]:
tickers.tickers.MSFT.info

{'zip': '98052-6399',
 'sector': 'Technology',
 'fullTimeEmployees': 163000,
 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. Its Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, and Skype for Business, as well as related Client Access Licenses (CAL); Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for small and medium businesses, large organizations, and divisions of enterprises. Its Intelligent Cloud segment licenses SQL and Windows Servers, Visual Studio, System Center, and related CALs; GitHub that provides a collaboration platform and code hosting service for developers; and Azure, a cloud platform. It also offers support services and Microsoft consulting services to assist customers in developing, deploying, and managing Microsoft server and de

In [13]:
from pandas_datareader import data as pdr
#import fix_yahoo_finance as yf
import yfinance as yf
from datetime import datetime

yf.pdr_override()
a = pdr.get_data_yahoo('MSFT', start=datetime(2017, 1, 1), end=datetime(2021, 2, 14))

[*********************100%***********************]  1 of 1 completed


In [15]:
msft = yf.Ticker('msft')

In [17]:
msftinfo = msft.info

In [24]:
print(msftinfo['shortName'])
print(msftinfo['sector'])
print(msftinfo['trailingPE'])
print(msftinfo['forwardPE'])
print(msftinfo['dividendRate'])
print(msftinfo['marketCap'])
print(msftinfo['longBusinessSummary'])

Microsoft Corporation
Technology
36.52751
30.283066
2.24
1847768514560
Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. Its Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, and Skype for Business, as well as related Client Access Licenses (CAL); Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for small and medium businesses, large organizations, and divisions of enterprises. Its Intelligent Cloud segment licenses SQL and Windows Servers, Visual Studio, System Center, and related CALs; GitHub that provides a collaboration platform and code hosting service for developers; and Azure, a cloud platform. It also offers support services and Microsoft consulting services to assist customers in developing, deploying, and managing Microsoft server and desktop solutions; and training a

In [28]:
df = pd.DataFrame(columns=['symbol','name','exchange','sector','trailing_pe','forward_pe','marketcap','summary'])

In [29]:
df = df.append({'symbol': 'msft',
                'name':msftinfo['shortName'],
                'exchange':'NASDAQ',
                'sector':msftinfo['sector'],
                'trailing_pe':msftinfo['trailingPE'],
                'forward_pe':msftinfo['forwardPE'],
                'marketcap':msftinfo['marketCap'],
                'summary':msftinfo['longBusinessSummary']}, 
               ignore_index=True)

In [30]:
df.head()

,symbol,name,exchange,sector,trailing_pe,forward_pe,marketcap,summary
0,msft,Microsoft Corporation,NASDAQ,Technology,36.52751,30.283066,1847768514560,"Microsoft Corporation develops, licenses, and ..."


In [42]:
_ticketinfo = yf.Ticker('AAPL').info

In [43]:
_ticketinfo['sector']

'Technology'